In [1]:
# !pip install 

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import tensorflow
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

[nltk_data] Downloading package punkt to /home/tejas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/tejas/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/tejas/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
2022-07-05 14:58:16.428722: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-07-05 14:58:16.461245: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-05 14:58:16.461258: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you d

In [2]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [3]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

33 documents
8 classes ['about', 'complaint', 'createaccount', 'goodbye', 'greeting', 'help', 'name', 'thanks']
52 unique lemmatized words ["'s", 'a', 'about', 'account', 'an', 'anyone', 'are', 'bye', 'call', 'can', 'complaint', 'could', 'create', 'do', 'for', 'give', 'goodbye', 'hand', 'have', 'hay', 'hello', 'help', 'helpful', 'hey', 'hi', 'how', 'i', 'is', 'later', 'me', 'name', 'need', 'new', 'open', 'please', 'raise', 'see', 'service', 'should', 'support', 'thank', 'thanks', 'that', 'the', 'there', 'to', 'want', 'what', 'whats', 'who', 'you', 'your']


In [4]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


/tmp/ipykernel_54732/2836083368.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [5]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")

Epoch 1/200


2022-07-05 14:58:31.016604: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-07-05 14:58:31.016622: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-05 14:58:31.016652: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tejas-ThinkPad-E15-Gen-2): /proc/driver/nvidia/version does not exist
2022-07-05 14:58:31.017074: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/tejas/odoo-v15/myenv/lib/python3.10/site-packages/keras/optimizers/optim

7/7 [==============================] - 0s 1ms/step - loss: 2.1386 - accuracy: 0.0909
Epoch 2/200
7/7 [==============================] - 0s 953us/step - loss: 2.0864 - accuracy: 0.0606
Epoch 3/200
7/7 [==============================] - 0s 1ms/step - loss: 2.0603 - accuracy: 0.1818
Epoch 4/200
7/7 [==============================] - 0s 983us/step - loss: 1.9024 - accuracy: 0.4242
Epoch 5/200
7/7 [==============================] - 0s 1ms/step - loss: 1.8614 - accuracy: 0.3030
Epoch 6/200
7/7 [==============================] - 0s 906us/step - loss: 1.8201 - accuracy: 0.3030
Epoch 7/200
7/7 [==============================] - 0s 1ms/step - loss: 1.7259 - accuracy: 0.3939
Epoch 8/200
7/7 [==============================] - 0s 904us/step - loss: 1.6339 - accuracy: 0.3333
Epoch 9/200
7/7 [==============================] - 0s 1ms/step - loss: 1.4971 - accuracy: 0.4848
Epoch 10/200
7/7 [==============================] - 0s 1ms/step - loss: 1.4512 - accuracy: 0.4848
Epoch 11/200
7/7 [===============

7/7 [==============================] - 0s 1ms/step - loss: 0.0376 - accuracy: 1.0000
Epoch 85/200
7/7 [==============================] - 0s 978us/step - loss: 0.0503 - accuracy: 1.0000
Epoch 86/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0719 - accuracy: 1.0000
Epoch 87/200
7/7 [==============================] - 0s 845us/step - loss: 0.0468 - accuracy: 1.0000
Epoch 88/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0700 - accuracy: 1.0000
Epoch 89/200
7/7 [==============================] - 0s 755us/step - loss: 0.0333 - accuracy: 1.0000
Epoch 90/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0547 - accuracy: 0.9697
Epoch 91/200
7/7 [==============================] - 0s 796us/step - loss: 0.0389 - accuracy: 1.0000
Epoch 92/200
7/7 [==============================] - 0s 1ms/step - loss: 0.1251 - accuracy: 0.9697
Epoch 93/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0771 - accuracy: 0.9697
Epoch 94/200
7/7 [=======

7/7 [==============================] - 0s 1ms/step - loss: 0.0114 - accuracy: 1.0000
Epoch 167/200
7/7 [==============================] - 0s 815us/step - loss: 0.0200 - accuracy: 1.0000
Epoch 168/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0302 - accuracy: 0.9697
Epoch 169/200
7/7 [==============================] - 0s 989us/step - loss: 0.0165 - accuracy: 1.0000
Epoch 170/200
7/7 [==============================] - 0s 894us/step - loss: 0.0186 - accuracy: 1.0000
Epoch 171/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0416 - accuracy: 1.0000
Epoch 172/200
7/7 [==============================] - 0s 842us/step - loss: 0.0055 - accuracy: 1.0000
Epoch 173/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0122 - accuracy: 1.0000
Epoch 174/200
7/7 [==============================] - 0s 823us/step - loss: 0.0130 - accuracy: 1.0000
Epoch 175/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0071 - accuracy: 1.0000
Epoch 176/200


In [6]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [7]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [8]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result
def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

In [ ]:
#Creating GUI with tkinter
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()

1/1 [==============================] - 0s 10ms/step
